In [3]:
import torch
from data.shapenet import ShapeNet
from model.vertix_model import VertixModel
import tqdm
from util.visualization import visualize_pointcloud, visualize_mesh
import random
%load_ext autoreload
%autoreload 2
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
num_vertices = 100

In [6]:
config = {
    'experiment_name': 'vertix_hungarian_train',
    'device': 'cuda:0',  
    'is_overfit': False,
    'batch_size': 8,
    'resume_ckpt': None,
    'learning_rate': 1e-3,
    'max_epochs': 5,
    'print_every_n': 100,
    'validate_every_n': 1000,
    'sdf_path': 'data/shapenet_dim32_sdf',
    'meshes_path': 'data/shapenet_reduced',
    'class_mapping': 'data/shape_info.json',
    'split': 'train',
    'num_vertices': num_vertices,
    'feature_size': 512,
    'num_trajectories': 2
}

In [7]:
dataset = ShapeNet(sdf_path=config["sdf_path"],
                         meshes_path=config["meshes_path"],
                         class_mapping=config["class_mapping"],
                         split = "overfit", threshold=config["num_vertices"], num_trajectories=config["num_trajectories"])


In [8]:
dataset.filter_data()

Length of dataset: 91
Filtering data ..


100%|█████████████████████████████████████████| 91/91 [00:00<00:00, 3493.91it/s]

Length of dataset: 10


In [9]:
from training import vertix_train

vertix_train.main(config)

Device: cuda:0
Length of dataset: 153528
Filtering data ..


100%|█████████████████████████████████| 153528/153528 [00:27<00:00, 5490.83it/s]


Length of dataset: 26196


100%|████████████████████████████████| 26196/26196 [00:00<00:00, 1156470.90it/s]


Class 03001627 has 4072 shapes
Class 04530566 has 2424 shapes
Class 04379243 has 6162 shapes
Class 03636649 has 1518 shapes
Class 02691156 has 2360 shapes
Class 02958343 has 3372 shapes
Class 04256520 has 4574 shapes
Class 02933112 has 1714 shapes
Length of dataset: 32304
Filtering data ..


100%|███████████████████████████████████| 32304/32304 [00:19<00:00, 1616.97it/s]


Length of dataset: 5474


100%|██████████████████████████████████| 5474/5474 [00:00<00:00, 1001728.63it/s]


Class 02691156 has 456 shapes
Class 04379243 has 1254 shapes
Class 02958343 has 700 shapes
Class 04256520 has 1016 shapes
Class 03636649 has 394 shapes
Class 02933112 has 360 shapes
Class 04530566 has 500 shapes
Class 03001627 has 794 shapes
[000/00099] train_loss: 0.102143
[000/00199] train_loss: 0.090373
[000/00299] train_loss: 0.086802
[000/00399] train_loss: 0.086395
[000/00499] train_loss: 0.083871
[000/00599] train_loss: 0.082954
[000/00699] train_loss: 0.078138
[000/00799] train_loss: 0.077803
[000/00899] train_loss: 0.077688
[000/00999] train_loss: 0.074215
[000/00999] val_loss: 0.082684 | best_loss_val: 0.082684
[000/01099] train_loss: 0.074380
[000/01199] train_loss: 0.073560
[000/01299] train_loss: 0.072092
[000/01399] train_loss: 0.068959
[000/01499] train_loss: 0.070014
[000/01599] train_loss: 0.067455
[000/01699] train_loss: 0.066268
[000/01799] train_loss: 0.066157
[000/01899] train_loss: 0.063957
[000/01999] train_loss: 0.065086
[000/01999] val_loss: 0.061445 | best_los

KeyboardInterrupt: 

In [7]:
from inference.inference_vertix import InferenceHandlerVertixModel

# create a handler for inference using a trained checkpoint
inferer = InferenceHandlerVertixModel('runs/vertix_hungarian_overfit/model_best.ckpt', config["num_vertices"])

In [8]:
x_indices = []
y_indices = []

graph = np.ones((1,config["num_vertices"], config["num_vertices"],1))

for i in range(config["num_vertices"]):
    for j in range(config["num_vertices"]):
        x_indices.append(i)
        y_indices.append(j)

In [9]:
random_sample = random.randint(0,len(dataset))

In [10]:
random_input = dataset[random_sample]["input_sdf"]
gt_pointcloud = dataset[random_sample]["target_vertices"]
gt_mask = dataset[random_sample]["input_mask"]
gt_edges = dataset[random_sample]["target_edges"]
#faces = dataset[random_sample]["faces"]

In [19]:
faces = []

for i in range(num_vertices):
    for j in range(num_vertices):
        for k in range(num_vertices):
            if gt_edges[i][j] and gt_edges[j][k]:
                faces.append(np.array([i,j,k]).reshape(1,-1))

In [11]:
faces = np.concatenate(faces,0)

NameError: name 'faces' is not defined

In [12]:
visualize_pointcloud(gt_pointcloud, point_size=0.01)

Output()

In [13]:
gt_pointcloud.shape

(150, 3)

In [14]:
visualize_mesh(gt_pointcloud, faces)

NameError: name 'faces' is not defined

In [15]:
output_pointcloud = inferer.infer_single(random_input)

In [25]:
faces = []

for i in range(num_vertices):
    for j in range(num_vertices):
        for k in range(num_vertices):
            if gt_edges[i][j] and gt_edges[j][k]:
                faces.append(np.array([i,j,k]).reshape(1,-1))
faces = np.concatenate(faces,0)

In [16]:
visualize_pointcloud(output_pointcloud,point_size=0.01)

Output()

In [27]:
target_size = int(gt_mask.sum())

cost = np.zeros((target_size, target_size))

for i in range(target_size):
    for j in range(target_size):
        distance = np.linalg.norm(gt_pointcloud[i] - output_pointcloud[j])
        cost[i][j] = distance

target_idx, vertix_idx = linear_sum_assignment(cost)

NameError: name 'linear_sum_assignment' is not defined

In [53]:
visualize_pointcloud(output_pointcloud[:int(gt_mask.sum())].detach().numpy().squeeze(),point_size=0.01)

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [19]:
edges = edges.detach().numpy().squeeze()

NameError: name 'edges' is not defined

In [20]:
faces = []

for i in range(num_vertices):
    for j in range(num_vertices):
        for k in range(num_vertices):
            if edges[i][j] and edges[j][k]:
                faces.append(np.array([i,j,k]).reshape(1,-1))

NameError: name 'edges' is not defined

In [21]:
faces = np.concatenate(faces,0)

ValueError: need at least one array to concatenate

In [28]:
visualize_mesh(output_pointcloud,faces)

Output()